In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, \
    classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
import os

# Load Data
df = pd.read_csv("C:/Users/User/Documents/CI 1/CA 2/diabetic_data.csv", dtype='category').iloc[:,2:]
df.shape

(101766, 48)

In [8]:
def show_unique(dataF):
    for c in dataF.columns:
        print(c, dataF[c].unique())
        print('*'*50)

show_unique(df)

race [Caucasian, AfricanAmerican, ?, Other, Asian, Hispanic]
Categories (6, object): [Caucasian, AfricanAmerican, ?, Other, Asian, Hispanic]
**************************************************
gender [Female, Male, Unknown/Invalid]
Categories (3, object): [Female, Male, Unknown/Invalid]
**************************************************
age [[0-10), [10-20), [20-30), [30-40), [40-50), [50-60), [60-70), [70-80), [80-90), [90-100)]
Categories (10, object): [[0-10), [10-20), [20-30), [30-40), ..., [60-70), [70-80), [80-90), [90-100)]
**************************************************
weight [?, [75-100), [50-75), [0-25), [100-125), [25-50), [125-150), [175-200), [150-175), >200]
Categories (10, object): [?, [75-100), [50-75), [0-25), ..., [125-150), [175-200), [150-175), >200]
**************************************************
admission_type_id [6, 1, 2, 3, 4, 5, 8, 7]
Categories (8, object): [6, 1, 2, 3, 4, 5, 8, 7]
**************************************************
discharge_disposition_

In [9]:
df.columns[df.isnull().any()]

Index([], dtype='object')

In [10]:
to_num = ['time_in_hospital', 'num_lab_procedures', 'num_procedures',
         'num_medications', 'number_outpatient', 'number_emergency',
         'number_inpatient', 'number_diagnoses']

to_cat_codes = list(set(df.columns) - set(to_num))

In [11]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

""
0
1
2
3
4


In [12]:
#setting the values for the diagnosis
    
def diag_num_update(x):    
    #circulatory= 1
    if 390 <= x and x <= 459 or x==785:
        return "1"
    
    #digestive= 2
    elif 520 <= x and x <= 579 or x==787:
        return "2"
        
    #genitourinary=3
    elif 580<= x and x <= 629 or x== 788:
        return "3"
    
    #diabetes=4
    elif 250.00<=x and x < 251:
        return "4"
    
    #injury=5
    elif 800<= x and x <=999:
        return "5"
    
    #musculoskeletal=6
    elif 710<= x and x <= 739:
        return "6"
    
    #neoplasms=7
    elif 140<= x and x<=239:
        return "7"
    
    #respiratory=8
    elif 460<= x and x<=519:
        return "8"
    
    #to cater for 0
    elif x==0:
        return "0"
    
    #for all others=9
    else:
        return "9"

In [13]:
# convert the column to numeric

df['diag_1']= df['diag_1'].apply(pd.to_numeric,errors='coerce')

In [14]:
df['diag_1']= df['diag_1'].apply(diag_num_update)

In [15]:
df['diag_1']= df['diag_1'].apply(pd.to_numeric,errors='coerce')

In [16]:
#do for diag_2

df['diag_2']= df['diag_2'].apply(pd.to_numeric,errors='coerce')

In [17]:
df['diag_2']= df['diag_2'].apply(diag_num_update)

In [18]:
df['diag_2']= df['diag_2'].apply(pd.to_numeric,errors='coerce')

In [19]:
#do for diag_3

df['diag_3']= df['diag_3'].apply(pd.to_numeric,errors='coerce')

In [20]:
df['diag_3']= df['diag_3'].apply(diag_num_update)

In [21]:
df['diag_3']= df['diag_3'].apply(pd.to_numeric,errors='coerce')

In [22]:
#if replace the duplicates in diag_1 to diag_3 with the value 0

df.loc[df.diag_1==df.diag_2,'diag_2']='0'

In [23]:
df.loc[df.diag_2==df.diag_3,'diag_3']='0'

In [24]:
df.loc[df.diag_1==df.diag_3,'diag_3']='0'

In [25]:
df = df.replace('?', np.nan)
df = df.replace('Unknown/Invalid', np.nan)
print(df.columns[df.isnull().any()])

Index(['race', 'gender', 'weight', 'payer_code', 'medical_specialty'], dtype='object')


In [26]:
#obj_df = df.select_dtypes(include=['object']).copy()
print(df.isnull().sum())
df = df.dropna(subset = ['gender']) # minimal na for gender, can drop rows
df = df.fillna({'weight':df['weight'].value_counts().index[0]}) # fill weight with the most common weight
df = df.fillna({'payer_code':'NOT_SPECIFIED', 'medical_specialty':'NOT_SPECIFIED'})
df.head()

race                         2273
gender                          3
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

ValueError: fill value must be in categories

In [28]:
df.loc[:,'New_diag1'] = '0'
df.loc[:,'New_diag2'] = '0'
df.loc[:,'New_diag3'] = '0'
df.loc[:,'New_diag4'] = '0'
df.loc[:,'New_diag5'] = '0'
df.loc[:,'New_diag6'] = '0'
df.loc[:,'New_diag7'] = '0'
df.loc[:,'New_diag8'] = '0'
df.loc[:,'New_diag9'] = '0'

In [29]:
#code out all the 1-9 values in diag1- diag3 into new columns



#make the new_diag 1 value 1 or 0
df.New_diag1 = np.where(df.diag_1 == 1,'1',df.New_diag1)
df.New_diag1 = np.where(df.diag_2 == 1,'1',df.New_diag1)
df.New_diag1 = np.where(df.diag_3 == 1,'1',df.New_diag1)

In [30]:
#make the new_diag 2 value 1 or 0
df.New_diag2 = np.where(df.diag_1 == 2, '1',df.New_diag2)
df.New_diag2 = np.where(df.diag_2 == 2, '1',df.New_diag2)
df.New_diag2 = np.where(df.diag_3 == 2, '1',df.New_diag2)

In [31]:
#make the new_diag 3 value 1 or 0
df.New_diag3 = np.where(df.diag_1 == 3,'1',df.New_diag3)
df.New_diag3 = np.where(df.diag_2 == 3,'1',df.New_diag3)
df.New_diag3 = np.where(df.diag_3 == 3,'1',df.New_diag3)

In [32]:
#make the new_diag 4 value 1 or 0
df.New_diag4 = np.where(df.diag_1 ==4, '1', df.New_diag4)
df.New_diag4 = np.where(df.diag_2 ==4, '1', df.New_diag4)
df.New_diag4 = np.where(df.diag_3 ==4, '1', df.New_diag4)

In [33]:
#make the new_diag 5 value 1 or 0
df.New_diag5 = np.where(df.diag_1 ==5, '1', df.New_diag5)
df.New_diag5 = np.where(df.diag_2 ==5, '1', df.New_diag5)
df.New_diag5 = np.where(df.diag_3 ==5, '1', df.New_diag5)

In [34]:
#make the new_diag 6 value 1 or 0
df.New_diag6 = np.where(df.diag_1 ==6, '1', df.New_diag6)
df.New_diag6 = np.where(df.diag_2 ==6, '1', df.New_diag6)
df.New_diag6 = np.where(df.diag_3 ==6, '1', df.New_diag6)

In [35]:
#make the new_diag 7 value 1 or 0
df.New_diag7 = np.where(df.diag_1 ==7, '1', df.New_diag7)
df.New_diag7 = np.where(df.diag_2 ==7, '1', df.New_diag7)
df.New_diag7 = np.where(df.diag_3 ==7, '1', df.New_diag7)

In [36]:
#make the new_diag 8 value 1 or 0
df.New_diag8 = np.where(df.diag_1 ==8, '1', df.New_diag8)
df.New_diag8 = np.where(df.diag_2 ==8, '1', df.New_diag8)
df.New_diag8 = np.where(df.diag_3 ==8, '1', df.New_diag8)

In [37]:
#make the new_diag 9 value 1 or 0
df.New_diag9 = np.where(df.diag_1 ==9, '1', df.New_diag9)
df.New_diag9 = np.where(df.diag_2 ==9, '1', df.New_diag9)
df.New_diag9 = np.where(df.diag_3 ==9, '1', df.New_diag9)

In [38]:
df.dtypes

race                        category
gender                      category
age                         category
weight                      category
admission_type_id           category
discharge_disposition_id    category
admission_source_id         category
time_in_hospital            category
payer_code                  category
medical_specialty           category
num_lab_procedures          category
num_procedures              category
num_medications             category
number_outpatient           category
number_emergency            category
number_inpatient            category
diag_1                         int64
diag_2                        object
diag_3                        object
number_diagnoses            category
max_glu_serum               category
A1Cresult                   category
metformin                   category
repaglinide                 category
nateglinide                 category
chlorpropamide              category
glimepiride                 category
a

In [39]:
df=df.drop('weight',axis=1)
df=df.drop('payer_code',axis=1)
df=df.drop('diag_1',axis=1)
df=df.drop('diag_2',axis=1)
df=df.drop('diag_3',axis=1)

In [40]:
df=df.drop('num_lab_procedures',axis=1)
df=df.drop('num_procedures',axis=1)
df=df.drop('number_emergency',axis=1)
df=df.drop('number_outpatient',axis=1)
df=df.drop('number_inpatient',axis=1)

In [41]:
df.dtypes

race                        category
gender                      category
age                         category
admission_type_id           category
discharge_disposition_id    category
admission_source_id         category
time_in_hospital            category
medical_specialty           category
num_medications             category
number_diagnoses            category
max_glu_serum               category
A1Cresult                   category
metformin                   category
repaglinide                 category
nateglinide                 category
chlorpropamide              category
glimepiride                 category
acetohexamide               category
glipizide                   category
glyburide                   category
tolbutamide                 category
pioglitazone                category
rosiglitazone               category
acarbose                    category
miglitol                    category
troglitazone                category
tolazamide                  category
e

In [42]:
#line to drop more columns
df=df.drop('num_medications',axis=1)
df=df.drop('number_diagnoses',axis=1)

In [43]:
#conversion to category

#for col in ['New_diag1','New_diag2','New_diag3','New_diag4','New_diag5','New_diag6','New_diag7','New_diag8','New_diag9',]:
    #df[col] = df[col].astype('category')

In [44]:
df.dtypes

race                        category
gender                      category
age                         category
admission_type_id           category
discharge_disposition_id    category
admission_source_id         category
time_in_hospital            category
medical_specialty           category
max_glu_serum               category
A1Cresult                   category
metformin                   category
repaglinide                 category
nateglinide                 category
chlorpropamide              category
glimepiride                 category
acetohexamide               category
glipizide                   category
glyburide                   category
tolbutamide                 category
pioglitazone                category
rosiglitazone               category
acarbose                    category
miglitol                    category
troglitazone                category
tolazamide                  category
examide                     category
citoglipton                 category
i

In [45]:
df['race'] = df['race'].astype('category')
df['race'] = df['race'].cat.codes
df['age'] = df['age'].astype('category')
df['age'] = df['age'].cat.codes
df['readmitted'] = df['readmitted'].astype('category')
df['readmitted'] = df['readmitted'].cat.codes

In [46]:
# one hot the rest
column_names = list(df.select_dtypes(include=['category']).columns.values)
one_hot = pd.get_dummies(df.select_dtypes(include=['category']))
df = df.drop(column_names, axis=1)
df = df.join(one_hot)

In [47]:
show_unique(df)

race [ 3  1 -1  5  2  4]
**************************************************
age [0 1 2 3 4 5 6 7 8 9]
**************************************************
readmitted [2 1 0]
**************************************************
New_diag1 ['0' '1']
**************************************************
New_diag2 ['0' '1']
**************************************************
New_diag3 ['0' '1']
**************************************************
New_diag4 ['1' '0']
**************************************************
New_diag5 ['0' '1']
**************************************************
New_diag6 ['0' '1']
**************************************************
New_diag7 ['0' '1']
**************************************************
New_diag8 ['0' '1']
**************************************************
New_diag9 ['0' '1']
**************************************************
gender_Female [1 0]
**************************************************
gender_Male [0 1]
*************************************************

acetohexamide_Steady [0 1]
**************************************************
glipizide_Down [0 1]
**************************************************
glipizide_No [1 0]
**************************************************
glipizide_Steady [0 1]
**************************************************
glipizide_Up [0 1]
**************************************************
glyburide_Down [0 1]
**************************************************
glyburide_No [1 0]
**************************************************
glyburide_Steady [0 1]
**************************************************
glyburide_Up [0 1]
**************************************************
tolbutamide_No [1 0]
**************************************************
tolbutamide_Steady [0 1]
**************************************************
pioglitazone_Down [0 1]
**************************************************
pioglitazone_No [1 0]
**************************************************
pioglitazone_Steady [0 1]
****************************

In [48]:
df_x = df.loc[:, df.columns != 'readmitted']
df_y = df.loc[:, df.columns == 'readmitted']
X_train, X_test, y_train, y_test = train_test_split(
    df_x, df_y, test_size=0.3, random_state=42)
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

In [1]:
for var_c in [5, 10, 20, 30, 40, 50, 80, 100]:
    clf = LinearSVC(C=var_c, dual=False).fit(X_train, y_train)
    print("C = " + str(var_c))
    print('training accuracy: {:.2f}'.format(clf.score(X_train, y_train)))
    print('test accuracy: {:.2f}'.format(clf.score(X_test, y_test)))

NameError: name 'LinearSVC' is not defined

In [50]:
from sklearn.svm import SVC


for var_c in [10, 20, 50, 80, 100]:
    clf = SVC(C=var_c, kernel='sigmoid').fit(X_train, y_train)
    print("C = " + str(var_c))
    print('training accuracy: {:.2f}'.format(clf.score(X_train, y_train)))
    print('test accuracy: {:.2f}'.format(clf.score(X_test, y_test)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [51]:
print('training accuracy: {:.2f}'.format(try1.score(X_train, y_train)))
print('test accuracy: {:.2f}'.format(try1.score(X_test, y_test)))

training accuracy: 0.54
test accuracy: 0.54


In [43]:
print(X_train)

            0         1    2    3    4    5    6    7    8    9   ...   225  \
0      0.333333  0.777778  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   1.0   
1      0.666667  0.888889  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 ...   1.0   
2      0.666667  0.666667  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0 ...   1.0   
3      0.666667  0.777778  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0 ...   1.0   
4      0.666667  0.666667  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 ...   1.0   
5      0.666667  0.666667  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0 ...   1.0   
6      0.666667  0.777778  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0 ...   1.0   
7      0.666667  0.555556  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   1.0   
8      0.666667  0.555556  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 ...   1.0   
9      0.666667  0.777778  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 ...   1.0   
10     0.666667  0.666667  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0 ...   1.0   
11     0.333333  0.888889  1.0  0.0  0.0  0.0  0.0  